# Cleaning and Merge File for Rocket Mortgage Classic and Weather Data
## Adie Maki 
## Final Project

## Import Libraries 

In [3]:
import pandas as pd
import numpy as np

## Clean the ESPN Rocket Mortgage Leaderboard

In [5]:
# load the data "espn_pga_leaderboard.csv" into pandas 
leader = pd.read_csv("espn_pga_leaderboard.csv")

# standardize the column names 
#  lowercase everything
#  remove leading and trailing spaces 
#  replace spaces with underscores 
#  remove puncuation
leader.columns = (
    leader.columns
    .str.strip()  # remove extra spaces 
    .str.lower()  # lowercase names 
    .str.replace(" ", "_")  # replace spaces with underscore 
    .str.replace(r"[^\w_]", "", regex=True))  # remove puncuation

# clean the player names 
#  strip spaces
#  remove punctuation 
leader["player"] = (
    leader["player"]
    .str.strip()
    .str.replace(r"[^\w\s\.-]", "", regex=True))  # allow letters, numbers, spaces, dots, and hyphens

# keep "score" as string for display purposes and readability 
#  (Ex: "-3", "E", "CUT")
leader["score_display"] = leader["score"]

# create a numeric score column 
#  remove none numeric words (CUT, WD, DQ, etc)
#  convert "E" (even par) to 0 
#  remove "+" signs 
#  convert to regular minus sign for under par rounds
leader["score_numeric"] = (
    leader["score"]
    .astype(str)
    .replace({"CUT":None,"WD":None,"DQ":None,"MC":None,"--":None})  # replace etxt codes with "None"
    .str.replace("E", "0", regex=False)  # convert "E" to 0
    .str.replace("+","", regex=False)  # remove plus signs
    .str.replace("−","-", regex=False))  # convert unicode minus sign

# convert score_numeric to actual numbers 
leader["score_numeric"] = pd.to_numeric(leader["score_numeric"], errors="coerce")

# Clean Round 1 - Round 4 and Total columns 
# remove the non numeric values 
# convert to floats 
round_cols = ["r1","r2","r3","r4","tot"]

for col in round_cols:
    # remove invalid markers 
    leader[col] = (
        leader[col]
        .astype(str)
        .replace({"CUT":None,"WD":None,"DQ":None,"MC":None,"--":None})
        .str.replace("+","", regex=False)  # remove plus signs 
        .str.replace("−","-", regex=False))  # fix the minus symbol
    # convert to numeric 
    leader[col] = pd.to_numeric(leader[col], errors="coerce")

# view the cleaned dataset
leader.head()


,unnamed_0,pos,player,score,r1,r2,r3,r4,tot,earnings,fedex_pts,score_display,score_numeric
0,NaN,1,T. Finau,-26,64.0,66.0,65.0,67.0,262.0,"$1,512,000",500,-26,-26.0
1,NaN,T2,P. Cantlay,-21,70.0,65.0,66.0,66.0,267.0,"$635,600",208,-21,-21.0
2,NaN,T2,C. Young,-21,71.0,63.0,65.0,68.0,267.0,"$635,600",208,-21,-21.0
3,NaN,T2,T. Pendrith,-21,64.0,65.0,66.0,72.0,267.0,"$635,600",208,-21,-21.0
4,NaN,5,S. Jaeger,-20,67.0,68.0,65.0,68.0,268.0,"$344,400",110,-20,-20.0


##  Clean the Tournament Weather Data

In [7]:
# load the Tournaments_Weather CSV file into pandas 
weather = pd.read_csv("Tournaments_Weather.csv")

# standardize the column names 
#  strip spaces 
#  lowercase 
#  replace spaces with underscores 
#  remove puncuation  
weather.columns = (
    weather.columns
        .str.strip()  # remove extra spaces 
        .str.lower()  # lowercase 
        .str.replace(" ", "_")  # use underscore instead of space 
        .str.replace(r"[^\w_]", "", regex=True))  # remove punctuation


# clean string columns 
#  remove extra whitespace
#  normalize capitalization
#  make text consistent 
#  remove punctuation 
string_cols = ["name", "course", "city", "state"]

for col in string_cols:
    weather[col] = (
        weather[col]
        .astype(str)  # make sure column is text 
        .str.strip()  # remove leading and trailing spaces 
        .str.replace(r"[^\w\s\.-]", "", regex=True))  # allow letters, numbers, spaces, dots 

# parse data column
#  converts data strings into datetime format
#  if parsing fails, pandas will assign NaT (missing data)
if "date" in weather.columns:
    weather["date"] = pd.to_datetime(weather["date"], errors="coerce")

# convert numeric columns 
#  pd.to_numeric(errors="coerce") turns bad data into NaN
numeric_cols = [
    "course_par","winning_score","cutline",
    "day0low","day0high","day0avg","day0precip","day0wind",
    "day1low","day1high","day1avg","day1precip","day1wind",
    "day2low","day2high","day2avg","day2precip","day2wind",
    "lat","long","year","id"]

for col in numeric_cols:
    if col in weather.columns:   # check columns exist 
        weather[col] = pd.to_numeric(weather[col], errors="coerce")

# output check 
print("Weather data cleaned, ready to merge.")
weather.head()


Weather data cleaned, ready to merge.


C:\Users\adiem\AppData\Local\Temp\ipykernel_68984\1436067499.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather["date"] = pd.to_datetime(weather["date"], errors="coerce")


,id,year,date,name,course,city,state,lat,long,course_par,...,day1high,day1avg,day1precip,day1wind,day2,day2low,day2high,day2avg,day2precip,day2wind
0,12014,2014,2013-10-10,Frys.com Open,CordeValle Golf Club - San Martin CA.,San Martin,CA,37.09,-121.59,71,...,78.80,64.40,0.000000,12.5459,10/11/2014,53.60,84.20,65.84,0.0,6.2154
1,12015,2015,2012-10-09,Frys.com Open,Silverado Resort and Spa North Course - Napa CA.,Napa,CA,38.30,-122.29,72,...,84.92,65.12,0.000000,9.2080,10/10/2015,57.20,77.00,64.04,0.0,11.0496
2,12016,2016,2018-10-15,Frys.com Open,Silverado Resort and Spa North Course - Napa CA.,Napa,CA,38.30,-122.29,72,...,69.08,61.52,0.311024,15.5385,10/16/2016,53.06,64.94,60.08,0.0,11.5100
3,12017,2017,2016-10-13,Safeway Open,Silverado Resort and Spa North Course - Napa CA.,Napa,CA,38.30,-122.29,72,...,73.94,53.78,0.000000,5.2946,10/14/2017,42.08,80.96,62.42,0.0,9.0929
4,12019,2019,2007-10-04,Safeway Open,Silverado Resort and Spa North Course - Napa CA.,Napa,CA,38.30,-122.29,72,...,73.94,57.56,0.000000,11.5100,10/5/2019,41.00,80.06,58.64,0.0,5.4097


## Begin Merge

In [9]:
# load both datasets from CSV files
# the leaderboard is the PGA Tour golfer results for the Rocket Mortgage tournament (2022)
# the weather is the weather conditions for all PGA Tournamnets (2009-2022)
leaderboard = pd.read_csv("espn_pga_leaderboard.csv")
weather = pd.read_csv("Tournaments_Weather.csv")

# filter weather to ONLY Rocket Mortgage Classic Tournament 
#  .str.contains("Rocket") finds any tournament name that includes "Rocket"
#  case=False makes the search case-sensitive
rocket_weather = weather[weather["name"].str.contains("Rocket", case=False)].copy()

# if the weather dataset has multiple years of Rocket Mortgage Tournament, this pulls only the most recent one (2022)
rocket_weather = rocket_weather.sort_values("year", ascending=False).head(1)

# reset indexes before merging 
#  ensures that each row will align cleanly when horizontally joining 
leaderboard = leaderboard.reset_index(drop=True)
rocket_weather = rocket_weather.reset_index(drop=True)

# drop the date column from weather if it exists 
#  the date of the tournament is not needed 
if "date" in rocket_weather.columns:
    rocket_weather = rocket_weather.drop(columns=["date"])

# duplicate the Rocket Mortgage weather row for each golfer count 
weather_expanded = pd.concat([rocket_weather] * len(leaderboard), ignore_index=True)

# final merge, horizontal join
merged = pd.concat([leaderboard, weather_expanded], axis=1)

# confirm merge was successful
print("Merge complete.")
merged.head()


# clean the round score for analysis 

# list of all round related columns that need to be numeric 
round_cols = ["R1", "R2", "R3", "R4", "TOT"]

# loop through each round column and clean 
for col in round_cols:
    # convert the column to string so replace() works 
    merged[col] = (
        merged[col]
        .astype(str)

        # replace all "Did Not Finish" with None
        .replace({"CUT": None, "WD": None, "DQ": None, "MC": None, "--": None})
        
        # remove any plus signs
        .str.replace("+", "", regex=False)

        # convert to a hyphen 
        .str.replace("−", "-", regex=False))
    
    # convert cleaned strings to numeric values, float 
    # invalid values will become NaN
    merged[col] = pd.to_numeric(merged[col], errors="coerce")

# confirmation 
print("Round scores cleaned for analysis.")


# clean the earnings column for analysis 

# some rows contain '--' which must be treated as missing
merged["EARNINGS"] = merged["EARNINGS"].replace({"--": None})

# remove dollar signs and commas
merged["EARNINGS_num"] = (
    merged["EARNINGS"]
    .astype(str)
    .str.replace(r"[\$,]", "", regex=True)) 

# convert to numeric 
merged["EARNINGS_num"] = pd.to_numeric(merged["EARNINGS_num"], errors="coerce")

print("Earnings cleaned, numeric column <EARNINGS_num> created.")



Merge complete.
Round scores cleaned for analysis.
Earnings cleaned, numeric column <EARNINGS_num> created.


## Create Score_Numeric 

In [11]:
# this converts the string score into a clean numeric column for analysis 

# clean the score column and prepare it to be converted to numeric 
merged["SCORE_NUMERIC"] = (
    merged["SCORE"]
    .astype(str)  # string 
    .replace({"CUT": None, "WD": None, "DQ": None, "MC": None}, regex=True)
    .str.replace("E", "0")  # even par
    .str.replace("−", "-", regex=False) # convert unicode minus
    .str.replace("+", "", regex=False))  # remove plus sign


# convert cleaned strings into actual numeric values, floats
# any remaining non numeric text becomes NaN
merged["SCORE_NUMERIC"] = pd.to_numeric(merged["SCORE_NUMERIC"], errors="coerce")

# print confirmation
print("SCORE_NUMERIC created successfully.")


SCORE_NUMERIC created successfully.


## Rename Columns

In [13]:
# rename weather columns to meaningful names for easier interpretation and clearly links the weather to the specific round

# make a copy of the merged dataset
analysis = merged.copy()

# rename the weather related columns using clear names
analysis = analysis.rename(columns={

    # pre-event (day before tournament, typically the practice round day)
    "day0low": "PreEvent_low",  # low temp 
    "day0high": "PreEvent_high",  # high temp
    "day0avg": "PreEvent_temp",  # avgerage temp
    "day0precip": "PreEvent_precip",  # precipitation 
    "day0wind": "PreEvent_wind",  # wind levels

    # round 1 weather
    "day1low": "R1_low",
    "day1high": "R1_high",
    "day1avg": "R1_temp",
    "day1precip": "R1_precip",
    "day1wind": "R1_wind",

    # round 2 weather
    "day2low": "R2_low",
    "day2high": "R2_high",
    "day2avg": "R2_temp",
    "day2precip": "R2_precip",
    "day2wind": "R2_wind"})

# confirm that the renaming worked
print("Weather columns renamed")
analysis.head()


Weather columns renamed


,Unnamed: 0,POS,PLAYER,SCORE,R1,R2,R3,R4,TOT,EARNINGS,...,R1_precip,R1_wind,day2,R2_low,R2_high,R2_temp,R2_precip,R2_wind,EARNINGS_num,SCORE_NUMERIC
0,NaN,1,T. Finau,-26,64.0,66.0,65.0,67.0,262.0,"$1,512,000",...,0.0,23.4804,7/29/2022,58.28,82.4,70.34,0.0,18.8764,1512000.0,-26.0
1,NaN,T2,P. Cantlay,-21,70.0,65.0,66.0,66.0,267.0,"$635,600",...,0.0,23.4804,7/29/2022,58.28,82.4,70.34,0.0,18.8764,635600.0,-21.0
2,NaN,T2,C. Young,-21,71.0,63.0,65.0,68.0,267.0,"$635,600",...,0.0,23.4804,7/29/2022,58.28,82.4,70.34,0.0,18.8764,635600.0,-21.0
3,NaN,T2,T. Pendrith,-21,64.0,65.0,66.0,72.0,267.0,"$635,600",...,0.0,23.4804,7/29/2022,58.28,82.4,70.34,0.0,18.8764,635600.0,-21.0
4,NaN,5,S. Jaeger,-20,67.0,68.0,65.0,68.0,268.0,"$344,400",...,0.0,23.4804,7/29/2022,58.28,82.4,70.34,0.0,18.8764,344400.0,-20.0


## Select only the needed variables for analysis and create a new csv file

In [15]:
# build the final dataset that will be used for analysis 

# select only the columns needed for analysis 
analysis_clean = analysis[[
    # player identifier 
    "PLAYER",
    # performance scores 
    "R1", "R2", "R3", "R4", "TOT",
    "SCORE_NUMERIC",

    # weather
    "PreEvent_temp", "PreEvent_precip", "PreEvent_wind",
    "R1_temp", "R1_wind", "R1_precip",
    "R2_temp", "R2_wind", "R2_precip",

    # performance outcomes
    "EARNINGS",
    "EARNINGS_num",
    "FEDEX PTS"]]

# export cleaned dataset for use in analysis
analysis_clean.to_csv("CleanedDataset_RocketMortgage.csv", index=False)

print("Final clean analysis dataset created: CleanedDataset_RocketMortgage.csv")

# show the first few rows 
analysis_clean.head()


Final clean analysis dataset created: CleanedDataset_RocketMortgage.csv


,PLAYER,R1,R2,R3,R4,TOT,SCORE_NUMERIC,PreEvent_temp,PreEvent_precip,PreEvent_wind,R1_temp,R1_wind,R1_precip,R2_temp,R2_wind,R2_precip,EARNINGS,EARNINGS_num,FEDEX PTS
0,T. Finau,64.0,66.0,65.0,67.0,262.0,-26.0,72.14,0.015748,11.1647,74.12,23.4804,0.0,70.34,18.8764,0.0,"$1,512,000",1512000.0,500
1,P. Cantlay,70.0,65.0,66.0,66.0,267.0,-21.0,72.14,0.015748,11.1647,74.12,23.4804,0.0,70.34,18.8764,0.0,"$635,600",635600.0,208
2,C. Young,71.0,63.0,65.0,68.0,267.0,-21.0,72.14,0.015748,11.1647,74.12,23.4804,0.0,70.34,18.8764,0.0,"$635,600",635600.0,208
3,T. Pendrith,64.0,65.0,66.0,72.0,267.0,-21.0,72.14,0.015748,11.1647,74.12,23.4804,0.0,70.34,18.8764,0.0,"$635,600",635600.0,208
4,S. Jaeger,67.0,68.0,65.0,68.0,268.0,-20.0,72.14,0.015748,11.1647,74.12,23.4804,0.0,70.34,18.8764,0.0,"$344,400",344400.0,110
